<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex09noteA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex09noteA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


----
## 準備
----

Google Colab の Notebook では， Python というプログラミング言語のコードを動かして計算したりグラフを描いたりできます．
Python は，機械学習・人工知能やデータサイエンスの分野ではメジャーなプログラミング言語ですが，それを学ぶことはこの授業の守備範囲ではありません．以下の所々に現れるプログラムっぽい記述の内容は，理解できなくて構いません．

以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

----
## データの前処理(1) 質的特徴量や欠損値の扱い
----






これまでのこの授業（「機械学習I」）で扱っていた例題では，与えられたデータをそのまま機械学習手法に適用していました．
しかし，実際のデータを扱う場合，入手可能な生のデータは，そのままでは機械学習に使えるような形式になっていないことがほとんどです．
そのような場合，自分で「下ごしらえ」して使える形式に整えてやらねばなりません．
この下ごしらえの処理のことを **前処理**（preprocessing）といいます．

この notebook と次の notebook では，よくある前処理の方法を解説します．




---
### 質的な特徴量の扱い

次のようなデータを考えてみましょう．

|番号|年齢|身長|好きなお菓子|好きな麺|服のサイズ|夏が好き？|合否|
|---:|---:|---:|:---:|:---:|:---:|---:|:---:|
|1|20|167.8|きのこの山|うどん|M|4|合|
|2|21|154.3|たけのこの里|中華麺|M|2|否|
|3|24|143.2|きのこの山|そば|S|5|合|
|4|31|198.7|きのこの山|うどん|XL|1|否|
|:|:|:|:|:|

このデータの各行は，ひとりひとりの人物の特徴を表しており，それぞれの特徴量は次のようなものとなっています：
- 年齢: 満年齢を表す0以上の整数値
- 身長: cm単位で表した身長の値
- 好きなお菓子: 「きのこの山」と「たけのこの里」のいずれか
- 好きな麺: 「うどん」「そば」「中華麺」のいずれか
- 服のサイズ: 「XS」「S」「M」 「L」「XL」のいずれか
- 夏が好き？: 1（大嫌い），2（嫌い），3（どちらでもない），4（好き），5（大好き）の5段階
- 合否: ある科目が合格だったか否か

このようなデータがあれば，例えば，「年齢」から「夏が好き？」までの特徴量から「合否」を予測する，という問題を考えることができます．識別問題の一種ですね．しかし，上記の特徴量には一部数値ではないものが含まれており，これまでに学んだ識別手法の多くではうまく扱えません．
そのような特徴量は，何らかの方法で扱える形に変換してやる必要があります．



#### 特徴量の分類

特徴量は，その性質によって，**量的**（**数値的**，**numerical**）なものと **質的**（**カテゴリカル**, **categorical**）なものに大別できます．



量的な特徴量は，整数や実数などの数値で表されたものです．上記の例では，「年齢」や「身長」が該当します．量的特徴量は，その値が離散(discrete)か連続(continuous)かでさらに二つに分けられます．
「年齢」は離散の，「身長」は連続な特徴量です（注）．

<span style="font-size: 75%">
注: もしも「年齢」を「生まれた瞬間からの経過時間」で表すなら連続量とみなせます．また，「身長」の値を適当な桁で四捨五入したりすれば離散量となります．
</span>

量的な特徴量とは，単に数値で表せるだけでなく，その値の「間隔」が等しいという性質も併せ持っているものを指します．「年齢」の値 20 と 21 の差は 50 と 51 の差とどちらも等しく「1歳の差」ですし，「身長」についても同様のことが言えます．

一方，「夏が好き？」は1から5の整数値で表されていますが，夏が好きな度合いを5段階で回答した値に過ぎず，1 と 2 の差が 2 と 3 の差と等しいとは限りません．したがって，これは量的な特徴量ではありません．


上記の例では，「好きなお菓子」「好きな麺」「服のサイズ」「夏が好き？」「合否」はいずれも質的な特徴量です．質的特徴量は，次の二つに分けられます．

<dl>
<dt><b>名義特徴量(nominal feature)</b></dt>
<dd>値同士に大小関係（順序）がなく，等しい値であるかどうかにのみ意味があるような特徴量．</dd>
<dt><b>順序特徴量(ordinal feature)</b></dt>
<dd>値同士に大小関係（順序）がある特徴量．ただし，↑で説明したように値の等間隔性はないもの．<dd>
</dl>

上記の例では，「好きなお菓子」「好きな麺」「合否」は名義特徴量，「服のサイズ」「夏が好き？」は順序特徴量と考えられます．
「服のサイズ」には XS < S < M < L < XL という順序が，「夏が好き？」には「大嫌い」 < 「嫌い」 < 「どちらでもない」 < 「好き」 < 「大好き」という順序があります（大小を逆に定義しても本質的には変わりません）．

#### 名義特徴量のダミー変数化／one-hot符号化

機械学習の多くの手法は，数値で表された特徴量しか扱えません（注）．
上記のように分類した特徴量のうち，量的なものについては値そのままでよいですが，質的なものは何らかの方法で数値として表現してやる必要があります．

<span style="font-size: 75%">
注: 決定木ベースの手法やその実装の中には，名義特徴量をそのまま扱えるものも存在します．
</span>

質的特徴量のうち順序特徴量は，値の順序を保って数値に変換することが容易です．
「服のサイズ」なら例えば $\textrm{XS} \rightarrow 1, \textrm{S} \rightarrow 2, \textrm{M} \rightarrow 3, \textrm{L} \rightarrow 4, \textrm{XL} \rightarrow 5$ とすればよいでしょう．
別に
$\textrm{XS} \rightarrow -0.2, \textrm{S} \rightarrow -0.1, \textrm{M} \rightarrow 0, \textrm{L} \rightarrow 0.1, \textrm{XL} \rightarrow 0.2$
でも構いませんし，数値の大小を逆にしても構いません．

一方，名義特徴量を数値に変換する際は注意が必要です．
例えば，「好きな麺」という特徴量は，「うどん」「そば」「中華麺」という3つの値をとります．
名義特徴量は「等しい値であるかどうかにのみ意味がある」ので，等しくないものはみな平等に扱わねばなりません．
しかし，これらを $\textrm{うどん}\rightarrow 1, \textrm{そば}\rightarrow 2, \textrm{中華麺}\rightarrow 3$ のように表したとすると，
「うどん」と「そば」および「そば」と「中華麺」の差が $1$ なのに，「うどん」と「中華麺」の差が $2$ となります．
これでは，「うどん」と「中華麺」は他の2種間よりも違いが大きいことになってしまいます．







上記の問題は，「好きな麺」を表す特徴量を
$$
\textrm{うどん}\rightarrow (1, 0, 0), \textrm{そば}\rightarrow (0, 1, 0), \textrm{中華麺}\rightarrow (0, 0, 1)
$$
のように3次元ベクトルとして表してやることで解消できます．
これならば，どの2つの値も等しく離れていることになります．
この例では，3次元ベクトルの最初の要素は「好きな麺」が「うどん」なら1でそれ以外なら0となります．
同様に，2番目の要素は「そば」なら1，3番目は「中華麺」なら1となります．
このようなやり方で名義特徴量を数値に変換することを，**ダミー変数化** や **one-hot符号化** といいます．



一般に，$K$通りの値を持つ名義特徴量は，$K$次元で，$0$か$1$のみを要素にもち，どれか1つの要素だけが$1$であるようなベクトルとして表します．
以下に，上記のデータのうち「好きなお菓子」と「好きな麺」をダミー変数化したものを示します．

|番号|好きなお菓子=きのこの山|好きなお菓子=たけのこの里|好きな麺=うどん|好きな麺=そば|好きな麺=中華麺|
|---:|:---:|:---:|:---:|:---:|:---:|
|1|1|0|1|0|0|
|2|0|1|0|0|1|
|3|1|0|0|1|1|
|4|1|0|1|0|0|
|:|:|:|:|:|

上記では，「好きなお菓子」を2列で，「好きな麺」を3列で表していますが，ダミー変数の性質上，1列を省略しても必要な情報が損なわれることはありません．たとえば，次のように表しても，2番目のひとの「好きなお菓子」が「たけのこの里」であり，「好きな麺」が「中華麺」であることは分かります．


|番号|好きなお菓子=きのこの山|好きな麺=うどん|好きな麺=そば|
|---:|:---:|:---:|:---:|
|1|1|1|0|
|2|0|0|0|
|3|1|0|1|
|4|1|1|0|
|:|:|:|:|:|

このように，$K$次元で表したダミー変数から1つの値を省略して$K-1$次元で表すこともあります．「好きなお菓子」や「合否」のように2通りの値しかとらない名義特徴量は，ほとんどの場合，2次元にはせず$0$か$1$の一つの値で表されます．

---
### 欠損値の扱い



以下は，「ほげ学」と「ふが学演習」という2科目の成績（0以上100以下の整数値）と，その成績をとったひとが「きのこの山」と「たけのこの里」のどちらが好きだったか，を調べて得られたデータです．
表の中に「-」というところがありますが，これは，そのひとがその科目を受講していなかったために成績がついていないことを表しています．

|番号|ほげ学|ふが学演習|きのたけ|
|---:|---:|:---:|:---:|
|1|95|82|きのこ|
|2|77|-|たけのこ|
|3|68|90|きのこ|
|4|-|88|たけのこ|
|5|78|100|きのこ|

実際のデータでは，このように一部の値が欠けている（**欠損値**(missing value)である）ことがよくあります．
欠損値は，測定・記録する器具の不具合や，そもそも値が存在しない（アンケートでその項目の回答がなかった，etc.）等，様々な理由で発生します．

欠損値を扱う際は，(1)欠損値をどのように表すか，(2)欠損値をどのように埋め合わせるか，の2つを主に考えることになります．

(1) 欠損値をどのように表すか

データを作成する際は，欠損値を何らかの方法で表さねばなりません．
そのための特別な仕掛けがなければ，普通の数値だけれども通常の値としては出現しないような値を使うことがよくあります（注）．
上記の成績の場合，欠損値を表す値として，例えば -1 や 999 のような値を使うことができます．

実際のデータでもこのような欠損値の表し方がよく使われますが，ある値が欠損値かどうかは数値を見ただけでは分かりません．そのため，欠損値と気づかずにそのまま使ってしまうというミスがおこりがちです．
どのような値が欠損値を表すかは場面によって様々ですので，注意深く処理することが必要です．


<span style="font-size: 75%">
注: Python の数値計算ライブラリ NumPy では，演算結果が不正であることを `np.nan` （nan は 'Not A Number' の略）という値で表します．データ分析ライブラリ pandas や機械学習ライブラリ scikit-learn の一部は，これを欠損値の印として扱うことができるようになっています．
</span>

In [ ]:
X = np.array([[95, 82], [77, np.nan], [68, 90], [np.nan, 88], [78, 100]])
print('np.sum:', np.sum(X, axis=0))  # np.sum は nan があると正しく和を計算できない
print('np.nansum:', np.nansum(X, axis=0)) # np.nansum は nan を無視して和を計算
df = pd.DataFrame(X, columns=['ほげ学', 'ふが学演習'])
print(df)
print(df.mean()) # 各列の平均

(2) 欠損値をどのように埋め合わせるか

機械学習の手法の多くは，欠損値の存在を考慮していません．
それらの手法を実装した機械学習ライブラリも，欠損値は扱えないことがほとんどです．
そのような場合，欠損値を含むデータを単に無視したり，欠損値を適当な値で埋めてやることになります．

後者の最も単純なやり方は，「欠損値ではない値の平均値（質的特徴量の場合は最頻値）を使う」というものです．
先の例では，「ほげ学」のその値は 79.5，「ふが学演習」のその値は 90.0 ですので，
2番目のひとの「ふが学演習」を 90.0 とし，3番目のひとの「ほげ学」を 79.5 とすることになります．
このやり方は簡単ですが，欠損値が複数出現しても全て同じ値で埋めてしまいます．
実用的にはもう少し凝ったやり方で欠損値を埋めることもありますが，ここではこれ以上は説明しません．
